In [2]:
import numpy as np
import cv2
import torch
from diffusers.utils import load_image
from diffusers import DiffusionPipeline
from diffusers import Flux2KleinPipeline

In [3]:
# Пути к фото-примерам из задания

autumn_pic_path = "Photo1.jpg"
summer_pic_path = "Photo2.jpg"

autumn = cv2.imread(autumn_pic_path)
summer = cv2.imread(summer_pic_path)

# Reinhart Color Transfer

In [4]:
def get_mean_and_std(x):
	x_mean, x_std = cv2.meanStdDev(x)
	x_mean = np.hstack(np.around(x_mean,2))
	x_std = np.hstack(np.around(x_std,2))
	return x_mean, x_std


def color_transfer(main, mask):
    main = cv2.cvtColor(main, cv2.COLOR_BGR2LAB)
    mask = cv2.cvtColor(mask, cv2.COLOR_BGR2LAB)

    main_mean, main_std = get_mean_and_std(main)
    mask_mean, mask_std = get_mean_and_std(mask)

    height, width, channel = main.shape

    for i in range(0, height):
        for j in range(0, width):
            for k in range(0, channel):
                x = main[i,j,k]
                x = ((x - main_mean[k]) * (mask_std[k] / main_std[k])) + mask_mean[k]
                x = round(x)
                x = 0 if x < 0 else x
                x = 255 if x > 255 else x
                main[i, j, k] = x

    main = cv2.cvtColor(main,cv2.COLOR_LAB2BGR)
    return main


In [ ]:
autumn_to_summer = color_transfer(autumn, summer)
summer_to_autumn = color_transfer(summer, autumn)

cv2.imwrite("autumn_to_summer.jpg", autumn_to_summer)
cv2.imwrite("summer_to_autumn.jpg", summer_to_autumn)

True

# FLUX.2-klein-4B

In [ ]:
# Если есть доступ к CUDA, лучше выполнить эту ячейку

device = 'cuda' if torch.cuda.is_available() else 'cpu'
pipe = DiffusionPipeline.from_pretrained(
    "black-forest-labs/FLUX.2-klein-4B", dtype=torch.bfloat16, 
    device_map=device
)

In [4]:
# Мне же пришлось запускать локально на MPS

pipe = Flux2KleinPipeline.from_pretrained(
    "black-forest-labs/FLUX.2-klein-4B", 
    torch_dtype=torch.float16,
)
pipe.to('mps')
pipe.enable_attention_slicing("max")

Loading pipeline components...: 100%|██████████| 5/5 [00:18<00:00,  3.69s/it]


## Превращение осени в лето
Картинки в оригинальном размере у меня не помещались, поэтому размеры были слегка уменьшены

In [5]:
prompt = "Change the season from autumn to summer. Replace all red and orange leaves with green summer foliage, vibrant grass. Keep the same trees, composition, and camera angle. Photorealistic landscape photography, Canon EOS R5, natural daylight, sharp details"
input_image = load_image(autumn_pic_path)
num_inference_steps = [10, 20, 25]

for num in num_inference_steps:
    image = pipe(image=input_image.resize((712, 712)), 
                prompt=prompt,
                num_inference_steps=num,
                guidance_scale=0.0).images[0]

    image.save(f"autumn_to_summer_flux2_{num}.png")

100%|██████████| 25/25 [08:49<00:00, 21.17s/it]


## Превращение лета в осень

In [7]:
prompt = "Change the season from summer to autumn. Replace all green summer leaves with vibrant red and orange autumn foliage with detailed veins and crisp edges, add scattered fallen leaves on the ground. Keep the exact same trees, composition, camera angle, and structure. Photorealistic landscape photography captured with Canon EOS R5, golden hour sunlight filtering through the canopy creating warm highlights on leaves, soft depth of field, sharp details, rich fall colors"
input_image = load_image(summer_pic_path)
num_inference_steps = [10, 20, 25]

for num in num_inference_steps:
    image = pipe(image=input_image.resize((int(input_image.size[0]*0.9), int(input_image.size[1]*0.9))), 
                prompt=prompt,
                num_inference_steps=num,
                guidance_scale=0.0).images[0]

    image.save(f"summer_to_autumn_flux2_{num}.png")

100%|██████████| 25/25 [09:43<00:00, 23.32s/it]
